# A quick description:

This is a script to query the SDSS Galaxy Zoo database for all or a number of galaxies that they are either ellipticals or spirals (according to their classification probability) and given a specific radius.
This version matches the GALFIT dataset where all galaxies are centered (but with real noise).

In [ ]:
# EDIT initial parameters here:

# Select if you want the query to return all galaxies or only a number 
# accepted values: 'all' or number (integer)

select_galaxies = 10      # 'all' | 10, 100, etc

# Determine the minimum probablity 
# for their classification as a spiral or elliptical
select_class_prob = 0.8

# Select min and max radius 
# (according to their Petrosian radius at 90% in the g band)
# value is in arcsec
select_min_radius = 10
select_max_radius = 20

# Size of cropped images (around the galaxy's position), in pixels
size = 128
hsize = size/2

In [ ]:
from astroquery.sdss import SDSS
# see details at: https://astroquery.readthedocs.io/en/latest/api/astroquery.sdss.SDSSClass.html#astroquery.sdss.SDSSClass.query_sql

if select_galaxies != 'all':
    query_selected_galaxies = "TOP {}".format(select_galaxies)
else:
    query_selected_galaxies = ""
    
queryZooGalaxies = """SELECT {}
zs.objid, zs.ra, zs.dec, zs.nvote,
zs.p_el,zs.p_cw, zs.p_acw, 
zs.p_edge, zs.p_dk, zs.p_mg, zs.p_cs,
p.objid, p.petroR90_g, p.ra,p.dec, p.run, p.rerun, p.camcol, p.field 
FROM ZooSpec AS zs
JOIN PhotoObj AS p ON p.objid = zs.objid
WHERE zs.p_cw+zs.p_acw > {} OR zs.p_el > {}
  AND p.petroR90_g > {} AND p.petroR90_g < {} """.format(query_selected_galaxies, select_class_prob,select_class_prob,select_min_radius,select_max_radius)

# uncomment that if you want to really see the sql query
#print queryZooGalaxies

resultsZoo = SDSS.query_sql(queryZooGalaxies)
print "> Number of objects returned: ", len(resultsZoo)

In [ ]:
import os, random, sys
import numpy as np
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

# clean the mess first...
os.system("rm *.fits *.png *.npy")

actual_sample = 0  # sample of galaxies used
spiral_sample = 0  # sample of spirals
ellipt_sample = 0  # sample of ellipticals

for i in range(0,len(resultsZoo)):
    objectID = resultsZoo[i][0]
    RA = resultsZoo[i][1]
    Dec = resultsZoo[i][2]
    runID = resultsZoo[i][-4]
    camcolID = resultsZoo[i][-2]
    fieldID = resultsZoo[i][-1]
    ellipt = float(resultsZoo[i][4])
    spiral = float(resultsZoo[i][5])+float(resultsZoo[i][6])

    print "[{}/{}] -- ID {}: ".format(i+1,len(resultsZoo), objectID),
    if ellipt>0.8:
        print "elliptical"
        name_root = "ellipt_{}".format(objectID)
    elif spiral>0.8:
        print "spiral"
        name_root = "spiral_{}".format(objectID)
    else:
        sys.exit("\n ERROR! Something is wrong with the classification")
        
    # getting g-band image from SDSS
    img = SDSS.get_images(run=runID, camcol=camcolID, field=fieldID, band='g')
    
    
    """
    # in case of debugging 
    print "image info --> "
    print img[0].info()   
    print img[0][0].header
    img_data = img[0][0].data
    print img_data
    # if you want to save the original image
    img[0].writeto(name_root+str('_original.fits'))
    """

    
    naxis1 = float(img[0][0].header['NAXIS1'])
    naxis2 = float(img[0][0].header['NAXIS2'])

    # WCS <-> pixel coordinate conversion
    w = WCS(img[0][0].header)
    x, y = w.wcs_world2pix(RA, Dec, 0)
    
    if x>5+hsize and y>5+hsize and np.abs(x-naxis1)>5+hsize and np.abs(x-naxis2)>5+hsize:
    # rejecting galaxies falling near the border of the image (cannot be cropped in a square)
    # NOTE: the +5 is arbitrary and compensate for roundings and for the conversion: pixel <-> matrix index
        cropped = img[0][0].data[int(y)-hsize:int(y)+hsize,int(x)-hsize:int(x)+hsize]
        np.save(name_root, cropped)
        print "\t cropping around WCS=(%s,%s) => pix=(%s,%s)" %  (RA, Dec, x, y)

        # updating samples
        actual_sample +=1
        if name_root.split('_')[0] == 'ellipt':
            ellipt_sample +=1
        elif name_root.split('_')[0] == 'spiral':
            spiral_sample +=1


        # selecting a random sample of galaxies to save as images 
        if random.random()>(1-100/len(resultsZoo)): 
            plt.imshow(cropped, cmap='gray', vmin=np.min(cropped), vmax=0.1*np.max(cropped), origin='lower')
            plt.savefig(name_root+'.png', bbox_inches='tight')
            fits.writeto(name_root+'.fits', cropped, img[0][0].header) 
            
    else:
        print "\t object at a corner - rejecting ... "
        
        
print "\n### Total Sums ###"
print "- Galaxies:\t", actual_sample
print "- Spirals:\t", spiral_sample
print "- Ellipticals:\t", ellipt_sample

# creating an image with all show cases
os.system('montage -label %f -tile 10x -geometry +0+0 *.png 00_showcase.jpg')        